# Athena audits: demo and state results
Show usage of the Audit class and demo on 2016 presidential contest in selected states

## Setup and define some utilities

In [1]:
from athena.audit import Audit
import math
import json
import sys

In [2]:
def make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule):
    "Convenience function to mak Audit class with given election parameters"

    election = {
        "alpha": alpha,
        "delta": delta,
        "candidates": candidates,
        "results": results,
        "ballots_cast": ballots_cast,
        "winners": winners,
        "name": name,
        "model": model,
        "pstop_goal": pstop_goal,
        "round_schedule": round_schedule,
    }
    a = Audit(audit_type, election['alpha'], election['delta'])
    a.add_election(election)
    return a

In [3]:
def find_next_round_size(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule):
    "Convenience function to call Audit class with given election parameters"

    election = {
        "alpha": alpha,
        "delta": delta,
        "candidates": candidates,
        "results": results,
        "ballots_cast": ballots_cast,
        "winners": winners,
        "name": name,
        "model": model,
        "pstop_goal": pstop_goal,
        "round_schedule": round_schedule,
    }
    a = Audit(audit_type, election['alpha'], election['delta'])
    a.add_election(election)
    x = a.find_next_round_size(election['pstop_goal'])
    return x

In [4]:
def sample90(margin, audit_type="ATHENA"):
    "Return sample size and other output given margin"

    assert 0.0 < margin < 1.0
    ballots_cast = 10000
    margin_votes = round(margin * ballots_cast)
    b = ballots_cast//2 - margin_votes // 2
    a = b + int(margin_votes)
    x = find_next_round_size(audit_type, 0.1, 1.0, ["A", "B"], [a, b], ballots_cast, 1, "state", "bin", [0.9], [])
    return (x['detailed']['A-B']['next_round_sizes'][0], x)

In [5]:
def sample90v(a, b, ballots_cast, audit_type="ATHENA"):
    "Return sample size etc. given votes for each of top two candidates in 1-winner contest"

    winner = max(a,b)
    loser = min(a,b)
    assert 0 < loser < winner < ballots_cast
    x = find_next_round_size(audit_type, 0.1, 1.0, ["A", "B"], [winner, loser], ballots_cast, 1, "state", "bin", [0.9], [])
    return (x['detailed']['A-B']['next_round_sizes'][0], x)

In [6]:
# Define a class to avoid cluttering notebook with stdout
class redirect_output(object):
    """context manager for reditrecting stdout/err to files"""


    def __init__(self, stdout='', stderr=''):
        self.stdout = stdout
        self.stderr = stderr

    def __enter__(self):
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr

        if self.stdout:
            sys.stdout = open(self.stdout, 'w')
        if self.stderr:
            if self.stderr == self.stdout:
                sys.stderr = sys.stdout
            else:
                sys.stderr = open(self.stderr, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr

# Basic demo of Audit class

In [7]:
audit_type = "ATHENA"
alpha = 0.1
delta = 1.0
candidates = ["A", "B"]
results = [600, 300]
ballots_cast = 1000
winners = 1
name = "test_election"
model = "bin"
pstop_goal = [.7, .8, .9]
round_schedule = []

In [8]:
a = make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [9]:
sample_size = 50
sample_results = [30]

In [10]:
a.add_round_schedule([sample_size])

In [11]:
a.round_schedule

[50]

In [12]:
x = a.find_risk(sample_results)



A (600) vs B (300)
	margin:	0.3333333333333333
{'kmins': [28], 'prob_sum': [0.7872329223739568], 'prob_tied_sum': [0.06757822542283723], 'deltas': [0.31280002486757374]}
find_kmins_for_risk
{'kmins': [30], 'passed': 1}

	AUDIT result:
		observed:	[30]
		required:	[28]

		Test passed

Delta:		0.5938312972095313
AUDIT risk:	0.11591596559800159


In [13]:
x

{'risk': 0.11591596559800159,
 'delta': 0.5938312972095313,
 'passed': 1,
 'observed': [30],
 'required': [28]}

In [14]:
x = find_next_round_size(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

setting round schedule


A (600) vs B (300)
	margin:	0.3333333333333333
	pstop goals: [0.7, 0.8, 0.9]
	scaled round schedule: []
		->	22	45
----	42	43	43	[43]	[0.7610118463711638]
42
	42	42	43	[42]	[0.6929293416775868]
		->	45	90
----	45	50	50	[50]	[0.8035861352110761]
45
	45	45	50	[45]	[0.7872329223739568]
	46	46	50	[46]	[0.753823829397414]
	48	48	50	[48]	[0.7799935744279759]
		->	45	90
----	56	62	62	[62]	[0.9021135361161704]
56
	56	56	62	[56]	[0.8609452974471495]
	57	57	62	[57]	[0.8375005329151851]
	58	58	62	[58]	[0.8762353612723259]
	59	59	62	[59]	[0.8548389798940872]
	60	60	62	[60]	[0.8899052715973235]
	61	61	62	[61]	[0.8704239984288676]
	63	63	62	[63]	[0.8844130736241462]
	65	65	62	[65]	[0.8969538871306307]
		0.7:	[47]	0.7610118463711638
		0.8:	[54]	0.8317821224884329
		0.9:	[73]	0.9227167435192974


In [15]:
x

{'detailed': {'A-B': {'pstop_goal': [0.7, 0.8, 0.9],
   'next_round_sizes': [47, 54, 73],
   'prob_stop': [0.7610118463711638, 0.8317821224884329, 0.9227167435192974]}},
 'future_round_sizes': [47, 54, 73]}

# Try to reproduce R2B2/Athena vs BRAVO
Sample Sizes for 90% probability of ending a Ballot Polling Audit of 2016 statewide Presidential contest

with risk limit 0.1, larger margins


In [16]:
# Read in data from 2016
election_2016 = json.load(open('data/2016_election.json'))

In [17]:
election_2016['Alabama']

{'contests': {'presidential': {'winners': 1,
   'candidates': ['Clinton', 'Trump'],
   'results': [729547, 1318255],
   'ballots_cast': 2123372,
   'state_id': 1,
   'margin': -0.2874828718792149}}}

In [18]:
def sample_state(state):
    "Return sample information for given state from 2016"

    election = election_2016[state]
    candidates = election['contests']['presidential']['candidates']
    results = election['contests']['presidential']['results']
    ballots_cast = election['contests']['presidential']['ballots_cast']
    athena_sample = sample90v(results[0], results[1], ballots_cast)
    return athena_sample

In [19]:
states = ['Alabama', 'Maryland', 'New York', 'Rhode Island', 'New Jersey', 'Ohio', 'Virginia',
          'Georgia', 'North Carolina', 'Arizona', 'Nevada' ]
# skip 'Minnesota', 'Florida', 'Wisconsin', 'Pennsylvania', 'Michigan']

In [ ]:
athena_results = {}
with redirect_output("debug_output.txt"):
  for state in states:
    athena_results[state] = sample_state(state)

In [ ]:
{s: r[0]  for s, r in athena_results.items()}

Why are these numbers all one less than those given in the recent diagram?

# Misc snippets of code

In [ ]:
with redirect_output("my_output.txt"):
    x = sample_state('Alabama')

In [ ]:
x

In [ ]:
e3 = find_next_round_size(audit_type, alpha, delta, ["A", "B", "C"], [600, 300, 100], ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [ ]:
e3

In [ ]:
# Earlier numbers
x = find_next_round_size(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [ ]:
x

In [ ]:
election = {
    "alpha": alpha,
    "delta": delta,
    "candidates": candidates,
    "results": results,
    "ballots_cast": ballots_cast,
    "winners": winners,
    "name": name,
    "model": model,
    "pstop": pstop_goal,
    "round_schedule": round_schedule,
    }

In [ ]:
e3 = {
    "alpha": alpha,
    "delta": delta,
    "candidates": ['A', 'B', 'C'],
    "results": results,
    "ballots_cast": ballots_cast,
    "winners": winners,
    "name": name,
    "model": model,
    "pstop": pstop_goal,
    "round_schedule": round_schedule,
    }